In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/BTC.csv', parse_dates= True, index_col=0)

In [3]:
result_df = pd.DataFrame(index=df.index)
result_df['close'] = df['close']
result_df.head(3)

,close
timestamp,
2018-01-01 00:00:00+00:00,13444.88
2018-01-02 00:00:00+00:00,14754.13
2018-01-03 00:00:00+00:00,15156.62


### Chaikin Oscillator

In [4]:
def chaikin_oscillator(df: pd.DataFrame, result_df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates the Chaikin Oscillator for a given DataFrame of financial data.

    Args:
        df (pd.DataFrame): DataFrame containing financial data, including columns 'close', 'high', 'low', and 'volume'.
        result_df (pd.DataFrame): DataFrame to store the results of the Chaikin Oscillator calculation.

    Returns:
        pd.DataFrame: The result DataFrame, shifted by one row.
    """
    # calculate the Chaikin Oscillator
    adl = ((2 * df['close'] - df['high'] - df['low']) / (df['high'] - df['low'])) * df['volume']
    adl_3_ema = adl.ewm(span=3).mean()
    adl_10_ema = adl.ewm(span=10).mean()
    chaikin_oscillator = adl_3_ema - adl_10_ema
    # Trading signals -> 1 indicates Buy, 0 indicates Hold and -1 indicates Sell
    result_df['Chaikin_oscillator'] = np.where(chaikin_oscillator > 0, 1, 0)
    result_df['Chaikin_oscillator'] = np.where(chaikin_oscillator < 0, -1, result_df['Chaikin_oscillator'])

    return result_df

result_df = chaikin_oscillator(df, result_df)
print(len(result_df.index))
result_df.Chaikin_oscillator.value_counts()

1920


 1    1000
-1     919
 0       1
Name: Chaikin_oscillator, dtype: int64

### Commodity Channel Index (CCI) 

In [5]:
def commodity_channel_index(df: pd.DataFrame, result_df: pd.DataFrame, period: int = 20) -> pd.DataFrame:
    """
    Calculates the Commodity Channel Index (CCI) for a given DataFrame of OHLC (open-high-low-close) prices.

    Args:
        df (pd.DataFrame): DataFrame of OHLC prices.
        result_df (pd.DataFrame): DataFrame to store the results.
        period (int, optional): The number of periods to use in the calculation. Defaults to 20.

    Returns:
        pd.DataFrame: The result DataFrame with the trading signals based on CCI values.
    """
    
    typical_price = (df['low'] + df['high'] + df['close']) / 3
    moving_average = typical_price.rolling(window=period).mean()
    mean_deviation = typical_price.rolling(window=period).apply(lambda x: np.fabs(x - x.mean()).mean())
    cci = (typical_price - moving_average) / (0.015 * mean_deviation)

    result_df['CCI'] = np.where(cci > 100, 1, 0)
    result_df['CCI'] = np.where(cci < -100, -1, result_df['CCI'])

    return result_df
result_df = commodity_channel_index(df,result_df)
print(len(result_df.index))
result_df.CCI.value_counts()

1920


 0    1127
 1     450
-1     343
Name: CCI, dtype: int64

### Williams Percent Range

In [6]:
def williams_r(df: pd.DataFrame,result_df, period: int = 14) -> pd.DataFrame:
    """
    Calculates the Williams %R indicator for a given DataFrame of OHLC (open-high-low-close) prices.

    Args:
        df (pd.DataFrame): DataFrame of OHLC prices.
        result_df (pd.DataFrame): DataFrame to store the results.
        period (int, optional): The number of periods to use in the calculation. Defaults to 14.

    Returns:
        pd.DataFrame: The result DataFrame with the  trading signals based on Williams %R values.
    """
    highest_high = df['high'].rolling(window = period).max()
    lowest_low = df['low'].rolling(window = period).min()
    wpr = -100 * (highest_high - df['close']) / (highest_high - lowest_low)

    result_df['williams_percentage'] = np.where(wpr < -80, 1, 0)
    result_df['williams_percentage'] = np.where(wpr > -20, -1, result_df['williams_percentage'])
    
    return result_df
result_df = williams_r(df,result_df)
print(len(result_df.index))
result_df.williams_percentage.value_counts()

1920


 0    1115
-1     475
 1     330
Name: williams_percentage, dtype: int64

### Parabolic SAR 

In [7]:
def parabolic_sar(df: pd.DataFrame, result_df: pd.DataFrame, af_start: float = 0.02, af_step: float = 0.02, af_max: float = 0.2):
    """
    Calculates the Parabolic SAR (Stop and Reverse) indicator for a given DataFrame of OHLC (open-high-low-close) prices.

    Args:
        df (pd.DataFrame): DataFrame of OHLC prices.
        result_df (pd.DataFrame): DataFrame to store the results.
        af_start (float, optional): Acceleration factor starting value. Defaults to 0.02.
        af_step (float, optional): Acceleration factor step. Defaults to 0.02.
        af_max (float, optional): Acceleration factor maximum value. Defaults to 0.2.

    Returns:
        pd.DataFrame: The result DataFrame with the trading signals based on PSAR values.
    """
    
    psar = df['high'].copy()
    psar[0] = df['low'][0]
    psar[1] = max(df['high'][0], df['high'][1])
    af = af_start
    trend = True
    af_list = []

    for i in range(2, len(df)):
        if trend:
            psar[i] = psar[i - 1] + af * (df['high'][i - 1] - psar[i - 1])
            if df['low'][i] < psar[i]:
                trend = False
                psar[i] = psar[i - 1]
                af_list.append(af)
                af = af_start
        else:
            psar[i] = psar[i - 1] + af * (df['low'][i - 1] - psar[i - 1])
            if df['high'][i] > psar[i]:
                trend = True
                psar[i] = psar[i - 1]
                af_list.append(af)
                af = af_start
        if trend:
            if df['high'][i] > df['high'][i - 1]:
                af = min(af + af_step, af_max)
        else:
            if df['low'][i] < df['low'][i - 1]:
                af = min(af + af_step, af_max)
        af_list.append(af)
    
    result_df['PSAR'] = np.where(df['close'] > psar, 1, 0)
    result_df['PSAR'] = np.where(df['close'] < psar, -1, result_df['PSAR'])
    
    
    return result_df

result_df = parabolic_sar(df,result_df)
print(len(result_df.index))
result_df.PSAR.value_counts()

1920


-1    974
 1    946
Name: PSAR, dtype: int64

### Money Flow Index (MFI)

In [8]:
def money_flow_index(df: pd.DataFrame, result_df: pd.DataFrame, n: int = 14) -> pd.DataFrame:
    """
    Calculates the Money Flow Index (MFI) for a given DataFrame of financial data.

    Args:
        df (pd.DataFrame): DataFrame containing financial data, including columns 'high', 'low', 'close', and 'volume'.
        result_df (pd.DataFrame): DataFrame to store the results of the MFI calculation.
        n (int, optional): The number of periods to use in the MFI calculation. Default is 14.

    Returns:
        pd.DataFrame: The result DataFrame.
    """
    typical_price = (df['high'] + df['low'] + df['close']) / 3
    raw_money_flow = typical_price * df['volume']
    positive_flow = np.where(typical_price > typical_price.shift(1), raw_money_flow, 0)
    negative_flow = np.where(typical_price < typical_price.shift(1), raw_money_flow, 0)
    positive_mf = pd.Series(positive_flow).rolling(n).sum()
    negative_mf = pd.Series(negative_flow).rolling(n).sum()
    mfi = 100 - (100 / (1 + (positive_mf / negative_mf)))
    mfi_signal = pd.Series(mfi).rolling(3).mean()
    result_df['MFI'] = np.where(mfi > mfi_signal, 1, 0)
    result_df['MFI'] = np.where(mfi < mfi_signal, -1, result_df['MFI'])
    return result_df

result_df = money_flow_index(df,result_df)
print(len(result_df.index))
result_df.MFI.value_counts()

1920


-1    955
 1    950
 0     15
Name: MFI, dtype: int64

### Keltner Channels

In [9]:
def keltner_channels(df: pd.DataFrame, result_df: pd.DataFrame, n: int = 10, m: int = 1) -> pd.DataFrame:
    """
    Calculates the Keltner Channels for a given DataFrame.

    Args:
        df (pandas.DataFrame): Input DataFrame containing 'open', 'high', 'low', and 'close' columns.
        result_df (pandas.DataFrame): DataFrame to store the results.
        n (int): Number of periods for calculating the EMA.
        m (int): Multiplier for calculating the upper and lower bands.
        
    Returns:
    pandas.DataFrame: DataFrame containing the trading signals.
    """
    
    middle_line = df['close'].ewm(span=n).mean()
    
    tr1 = df['high'] - df['low']
    tr2 = abs(df['high'] - df['close'].shift())
    tr3 = abs(df['low'] - df['close'].shift())
    true_range = pd.DataFrame({'tr1': tr1, 'tr2': tr2, 'tr3': tr3}).max(axis=1)
    
    upper_band = middle_line + m * true_range.ewm(span=n).mean()
    lower_band = middle_line - m * true_range.ewm(span=n).mean()

    
    # Generate signals
    result_df['Keltner_channels'] = 0
    result_df.loc[(df['close'] > upper_band), 'Keltner_channels'] = -1  # Sell signal
    result_df.loc[(df['close'] < lower_band), 'Keltner_channels'] = 1  # Buy signal
    
    return result_df
result_df = keltner_channels(df,result_df)
print(len(result_df.index))
result_df.Keltner_channels.value_counts()

1920


 0    1382
-1     327
 1     211
Name: Keltner_channels, dtype: int64

### Trix Indicator

In [10]:
def trix(df: pd.DataFrame, result_df: pd.DataFrame, n: int=15, m: int=9) -> pd.DataFrame:
    """
    Calculates the TRIX indicator and trading signals for a given DataFrame.

    Args:
        df (pandas.DataFrame): Input DataFrame containing 'open', 'high', 'low', and 'close' columns.
        result_df (pandas.DataFrame, optional): DataFrame to store the results. If not provided, a new DataFrame will be created.
        n (int, optional): Number of periods for calculating the TRIX.
        m (int, optional): Number of periods for calculating the signal line.

    Returns:
    pandas.DataFrame: DataFrame containing the trading signals.
    """

    ema = df['close'].ewm(span=n, min_periods=n).mean()
    ema_ema = ema.ewm(span=n, min_periods=n).mean()
    trix = (ema_ema - ema_ema.shift(1)) / ema_ema.shift(1) * 100
    trix_signal = trix.ewm(span=m, min_periods=m).mean()

    # Trading signals
    result_df['TRIX'] = np.where(trix > trix_signal, 1, 0)
    result_df['TRIX'] = np.where(trix < trix_signal, -1, result_df['TRIX'])
    
    return result_df


result_df = trix(df,result_df)
print(len(result_df.index))
result_df.TRIX.value_counts()

1920


 1    963
-1    920
 0     37
Name: TRIX, dtype: int64

### Ultimate Oscillator

In [11]:
def ultimate_oscillator(df, result_df, short_period=7, medium_period=14, long_period=28, buy_thresh=30, sell_thresh=70):
    """
    This function calculates the Ultimate Oscillator for a given DataFrame and generates buy/sell signals based on the oscillator values.

    Args:
    df (pandas.DataFrame): Input DataFrame containing 'open', 'high', 'low', and 'close' columns.
    result_df (pandas.DataFrame): DataFrame to store the results.
    short_period (int): Number of periods for calculating the short-term average.
    medium_period (int): Number of periods for calculating the medium-term average.
    long_period (int): Number of periods for calculating the long-term average.
    buy_thresh (float): Buy threshold for the oscillator.
    sell_thresh (float): Sell threshold for the oscillator.

    Returns:
    pandas.DataFrame: DataFrame containing the Ultimate Oscillator and signals.
    """

    # Calculate buying pressure, true range, and average true range
    bp = df['close'] - pd.concat([df['low'], df['close'].shift(1)], axis=1).min(axis=1)
    tr = pd.concat([df['high'] - df['low'], abs(df['high'] - df['close'].shift(1)), abs(df['low'] - df['close'].shift(1))], axis=1).max(axis=1)
    atr_short = tr.rolling(short_period).sum()
    atr_medium = tr.rolling(medium_period).sum()
    atr_long = tr.rolling(long_period).sum()

    # Calculate the buying pressure for each period
    bp_short = bp.rolling(short_period).sum()
    bp_medium = bp.rolling(medium_period).sum()
    bp_long = bp.rolling(long_period).sum()

    # Calculate the raw Ultimate Oscillator values
    raw_uo_short = bp_short / atr_short
    raw_uo_medium = bp_medium / atr_medium
    raw_uo_long = bp_long / atr_long

    # Calculate the weighted Ultimate Oscillator values
    total_period = short_period + medium_period + long_period
    wuo_short = (4 * raw_uo_short + 2 * raw_uo_medium + raw_uo_long) / 7
    wuo_medium = (2 * raw_uo_short + 4 * raw_uo_medium + raw_uo_long) / 7
    wuo_long = (raw_uo_short + 2 * raw_uo_medium + 4 * raw_uo_long) / 7

    # Calculate the Ultimate Oscillator
    uo = (wuo_short * short_period + wuo_medium * medium_period + wuo_long * long_period) / total_period

    # Trading signals
    result_df['Ultimate_oscillator'] = np.where(uo > sell_thresh, -1, 0)
    result_df['Ultimate_oscillator'] = np.where(uo < buy_thresh, 1, result_df['Ultimate_oscillator'])

    return result_df


result_df = ultimate_oscillator(df,result_df)
print(len(result_df.index))
result_df.Ultimate_oscillator.value_counts()

1920


1    1893
0      27
Name: Ultimate_oscillator, dtype: int64

### Standard Deviation


In [12]:
def ad_indicator(df, result_df, window_short = 3, window_long = 10):
    """This function calculates the Accumulation/Distribution (A/D) indicator for a given DataFrame.

    Args:
    df (pandas.DataFrame): Input DataFrame containing 'open', 'high', 'low', 'close', and 'volume' columns.
    result_df (pandas.DataFrame): DataFrame to store the results.

    Returns:
    pandas.DataFrame: DataFrame containing the A/D indicator and signals.
    """
    ad = ((df['close'] - df['low']) - (df['high'] - df['close'])) / (df['high'] - df['low']) * df['volume']
    ad = ad.cumsum()

    ad_ma_3 = ad.rolling(window_short).mean()
    ad_ma_10 = ad.rolling(window_long).mean()

    # Trading signals
    result_df['ad_indicator'] = np.where(ad_ma_3 > ad_ma_10, 1, 0)
    result_df['ad_indicator'] = np.where(ad_ma_3 < ad_ma_10, -1, result_df['ad_indicator'])
    
    return result_df


result_df = ad_indicator(df, result_df)
print(len(result_df.index))
result_df.ad_indicator.value_counts()


1920


 1    1172
-1     739
 0       9
Name: ad_indicator, dtype: int64

### Donchain Channels

In [13]:
def donchian_channel(df, result_df, n = 20):
    """
    This function calculates the Donchian Channel for a given DataFrame.

    Parameters:
    df (pandas.DataFrame): Input DataFrame containing 'open', 'high', 'low', and 'close' columns.
    result_df (pandas.DataFrame): DataFrame to store the results.
    n (int): Number of periods for calculating the Donchian Channel.

    Returns:
    pandas.DataFrame: DataFrame containing the Donchian Channel signals.
    """
    upper_band = df['high'].rolling(n).max()
    lower_band = df['low'].rolling(n).min()
    middle_band = (upper_band + lower_band) / 2

    # Trading signals
    result_df['Donchian_channel'] = np.where(df['close'] > upper_band, -1, 0)
    result_df['Donchian_channel'] = np.where(df['close'] < lower_band, 1, result_df['Donchian_channel'])
    result_df['Donchian_channel'] = np.where((df['close'] < middle_band) & (df['close'].shift(1) > middle_band.shift(1)), 1, result_df['Donchian_channel'])
    result_df['Donchian_channel'] = np.where((df['close'] > middle_band) & (df['close'].shift(1) < middle_band.shift(1)), -1, result_df['Donchian_channel'])
    
    return result_df

result_df = donchian_channel(df, result_df)
print(len(result_df.index))
result_df.Donchian_channel.value_counts()

1920


 0    1731
-1      95
 1      94
Name: Donchian_channel, dtype: int64

### SMA Crossover

In [14]:
def sma_crossover(df,result_df, short_period = 30, long_period = 60):
    sma_short = df['close'].rolling(window=short_period).mean()
    sma_long = df['close'].rolling(window=long_period).mean()
    result_df['SMA'] = 0
    result_df.loc[sma_short > sma_long, 'SMA'] = 1
    result_df.loc[sma_short < sma_long, 'SMA'] = -1
    return result_df

result_df = sma_crossover(df, result_df)
print(len(result_df.index))
result_df.SMA.value_counts()

1920


-1    943
 1    918
 0     59
Name: SMA, dtype: int64

### EMA Crossover

In [15]:
def ema_crossover(df,result_df, short_period = 30, long_period = 60):
    ema_short = df['close'].ewm(span=short_period, adjust=False).mean()
    ema_long = df['close'].ewm(span=long_period, adjust=False).mean()
    result_df['EMA'] = 0
    result_df.loc[ema_short > ema_long, 'EMA'] = 1
    result_df.loc[ema_short < ema_long, 'EMA'] = -1
    return result_df
result_df = ema_crossover(df, result_df)
print(len(result_df.index))
result_df.EMA.value_counts()

1920


-1    1035
 1     884
 0       1
Name: EMA, dtype: int64

## RSI

In [16]:
def rsi_strategy(df,result_df, rsi_period=14, rsi_upper=70, rsi_lower=30):
    delta = df['close'].diff()
    up = delta.where(delta > 0, 0)
    down = -delta.where(delta < 0, 0)
    avg_gain = up.rolling(rsi_period).mean()
    avg_loss = down.rolling(rsi_period).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    result_df['RSI'] = 0
    result_df['RSI'] = np.where(rsi > rsi_upper, -1, 0)
    result_df['RSI'] = np.where(rsi < rsi_lower, 1, result_df['RSI'])

    return result_df
result_df = rsi_strategy(df, result_df)
print(len(result_df.index))
result_df.RSI.value_counts()

1920


 0    1379
-1     303
 1     238
Name: RSI, dtype: int64

### Ichimoku Cloud

In [17]:
def ichimoku_cloud_signals(df, result_df):
    period9_high = df['high'].rolling(window=9).max()
    period9_low = df['low'].rolling(window=9).min()
    conversion_line = (period9_high + period9_low) / 2
    period26_high = df['high'].rolling(window=26).max()
    period26_low = df['low'].rolling(window=26).min()
    base_line = (period26_high + period26_low) / 2
    spanA = ((conversion_line + base_line) / 2).shift(26)
    period52_high = df['high'].rolling(window=52).max()
    period52_low = df['low'].rolling(window=52).min()
    spanB = ((period52_high + period52_low) / 2).shift(26)
    chikou_span = df['close'].shift(-26)
    buy_signal = (spanA > spanB) & (df['close'] > spanA) & (df['close'] > spanB) & (df['close'] > chikou_span)
    sell_signal = (spanA < spanB) & (df['close'] < spanA) & (df['close'] < spanB) & (df['close'] < chikou_span)
    result_df['ichimoku_cloud'] = 0
    result_df.loc[buy_signal, 'ichimoku_cloud'] = 1
    result_df.loc[sell_signal, 'ichimoku_cloud'] = -1
    return result_df
result_df = ichimoku_cloud_signals(df, result_df)
print(len(result_df.index))
result_df.ichimoku_cloud.value_counts()

1920


 0    1391
-1     315
 1     214
Name: ichimoku_cloud, dtype: int64

### MACD Strategy

In [18]:
def macd_strategy(data,result_df, fast_period=12, slow_period=26, signal_period=9):

    ema_fast = df['close'].ewm(span=fast_period, adjust=False).mean()
    ema_slow = df['close'].ewm(span=slow_period, adjust=False).mean()
    macd = ema_fast - ema_slow
    signal = macd.ewm(span=signal_period, adjust=False).mean()
    # Generate trading signals based on the MACD and signal line
    result_df['MACD'] = 0
    result_df['MACD'] = np.where(macd > signal, 1, 0)
    result_df['MACD'] = np.where(macd < signal, -1, result_df['MACD'])
    return result_df
result_df = macd_strategy(df, result_df)
print(len(result_df.index))
result_df.MACD.value_counts()

1920


 1    1024
-1     895
 0       1
Name: MACD, dtype: int64

### Stochastic Oscillator Strategy

In [19]:
def stochastic_oscillator_strategy(df, result_df, period=14, oversold=20, overbought=80):
    highs = df['high']
    lows = df['low']
    closes = df['close']

    # Calculate the stochastic oscillator
    stochastic_oscillator = 100 * ((closes - lows.rolling(period).min()) / (highs.rolling(period).max() - lows.rolling(period).min()))
    result_df['Stochastic_oscillator'] = 0
    result_df['Stochastic_oscillator'] = np.where(stochastic_oscillator < oversold, 1, 0)
    result_df['Stochastic_oscillator'] = np.where(stochastic_oscillator > overbought, -1, result_df['Stochastic_oscillator'])

    return result_df
result_df = stochastic_oscillator_strategy(df, result_df)
print(len(result_df.index))
result_df.Stochastic_oscillator.value_counts()

1920


 0    1115
-1     475
 1     330
Name: Stochastic_oscillator, dtype: int64

### Volume Weighted Average Price (VWAP)

In [20]:
def vwap_signals(df, result_df):
    df['TP'] = (df['high'] + df['low'] + df['close']) / 3
    df['VP'] = df['TP'] * df['volume']
    df['Cumulative VP'] = df['VP'].cumsum()
    df['Cumulative Volume'] = df['volume'].cumsum()
    df['VWAP'] = df['Cumulative VP'] / df['Cumulative Volume']

    buy_signal = df['close'] > df['VWAP']
    sell_signal = df['close'] < df['VWAP']
    result_df['Volume_Weighted_Average_Price'] = 0
    result_df.loc[buy_signal, 'Volume_Weighted_Average_Price'] = 1
    result_df.loc[sell_signal, 'Volume_Weighted_Average_Price'] = -1
    return result_df
result_df = vwap_signals(df, result_df)
print(len(result_df.index))
result_df.Volume_Weighted_Average_Price.value_counts()


1920


-1    998
 1    922
Name: Volume_Weighted_Average_Price, dtype: int64

### Bollinger Band Strategy

In [21]:
def bollinger_band_strategy(df, result_df, window_size=20, num_std=2):
    rolling_mean = df['close'].rolling(window=window_size).mean()
    rolling_std = df['close'].rolling(window=window_size).std()
    upper_band = rolling_mean + (rolling_std * num_std)
    lower_band = rolling_mean - (rolling_std * num_std)
    result_df['Bollinger_band'] = 0
    buy = np.where(df['close'] < lower_band, 1, 0)
    sell = np.where(df['close'] > upper_band, -1, 0)
    result_df['Bollinger_band'] = buy + sell
    return result_df
result_df = bollinger_band_strategy(df, result_df)
print(len(result_df.index))
result_df.Bollinger_band.value_counts()

1920


 0    1711
-1     123
 1      86
Name: Bollinger_band, dtype: int64

### Fibonacci Retracements Strategy

In [22]:
def fibonacci_retracements_strategy(df, result_df, buy_level=0.618, sell_level=0.382):
    close_prices = df['close']
    high = max(close_prices)
    low = min(close_prices)
    diff = high - low
    levels = [low + buy_level * diff, low + sell_level * diff]
    signals = [0] * len(close_prices)
    for i in range(len(close_prices)):
        if close_prices[i] <= levels[0]:
            signals[i] = 1
        elif close_prices[i] >= levels[1]:
            signals[i] = -1
    result_df['Fibonacci_retracement'] = signals
    return result_df
result_df = fibonacci_retracements_strategy(df, result_df)
print(len(result_df.index))
result_df.Fibonacci_retracement.value_counts()

1920


 1    1647
-1     273
Name: Fibonacci_retracement, dtype: int64

### ADX Strategy

In [23]:
def adx_strategy(df, result_df, n=14, adx_threshold=25):

    df['tr'] = np.max([df['high'] - df['low'], abs(df['high'] - df['close'].shift()), abs(df['low'] - df['close'].shift())], axis=0)
    df['up_move'] = df['high'] - df['high'].shift()
    df['down_move'] = df['low'].shift() - df['low']
    df['up_move'] = np.where((df['up_move'] > df['down_move']) & (df['up_move'] > 0), df['up_move'], 0)
    df['down_move'] = np.where((df['up_move'] < df['down_move']) & (df['down_move'] > 0), df['down_move'], 0)
    df['atr'] = df['tr'].rolling(window=n).mean()
    df['plus_di'] = 100 * (df['up_move'] / df['atr']).ewm(span=n, adjust=False).mean()
    df['minus_di'] = 100 * (df['down_move'] / df['atr']).ewm(span=n, adjust=False).mean()
    df['dx'] = 100 * (abs(df['plus_di'] - df['minus_di']) / (df['plus_di'] + df['minus_di'])).ewm(span=n, adjust=False).mean()
    df['adx'] = df['dx'].ewm(span=n, adjust=False).mean()
    
    
    result_df['ADX'] = np.where((df['adx'] > adx_threshold) & (df['plus_di'] > df['minus_di']), 1, np.nan)
    result_df['ADX'] = np.where((df['adx'] > adx_threshold) & (df['plus_di'] < df['minus_di']), -1, result_df['ADX'])
    df.drop(['tr', 'up_move', 'down_move', 'atr', 'dx'], axis=1, inplace=True)

    return result_df
result_df = adx_strategy(df, result_df)
print(len(result_df.index))
result_df.ADX.value_counts()

1920


-1.0    910
 1.0    886
Name: ADX, dtype: int64

In [24]:
result_df.tail(10)

,close,Chaikin_oscillator,CCI,williams_percentage,PSAR,MFI,Keltner_channels,TRIX,Ultimate_oscillator,ad_indicator,...,SMA,EMA,RSI,ichimoku_cloud,MACD,Stochastic_oscillator,Volume_Weighted_Average_Price,Bollinger_band,Fibonacci_retracement,ADX
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-03-26 00:00:00+00:00,27996.81,1,0,-1,1,-1,0,1,1,-1,...,1,1,-1,0,1,-1,-1,0,1,1.0
2023-03-27 00:00:00+00:00,27145.09,-1,0,0,1,-1,0,1,1,-1,...,1,1,0,0,1,0,-1,0,1,1.0
2023-03-28 00:00:00+00:00,27274.90,1,0,0,1,-1,0,-1,1,1,...,1,1,0,0,1,0,-1,0,1,1.0
2023-03-29 00:00:00+00:00,28355.87,1,0,-1,1,1,0,-1,1,1,...,1,1,0,0,1,-1,-1,0,1,1.0
2023-03-30 00:00:00+00:00,28037.46,-1,0,0,1,1,0,-1,1,1,...,1,1,0,0,1,0,-1,0,1,1.0
2023-03-31 00:00:00+00:00,28477.29,1,0,0,1,-1,0,-1,1,1,...,1,1,0,0,1,0,-1,0,1,1.0
2023-04-01 00:00:00+00:00,28465.30,-1,0,0,1,-1,0,-1,1,1,...,1,1,0,0,-1,0,-1,0,1,1.0
2023-04-02 00:00:00+00:00,28186.76,-1,0,0,1,-1,0,-1,1,1,...,1,1,0,0,-1,0,-1,0,1,1.0
2023-04-03 00:00:00+00:00,27810.08,-1,0,0,1,-1,0,-1,1,1,...,1,1,0,0,-1,0,-1,0,1,1.0


In [25]:
result_df['return'] = result_df['close'].pct_change(periods=3).mul(100)
result_df.dropna(inplace=True)
result_df['target'] = 0
result_df.loc[result_df['return'] > 2, 'target'] = 1
result_df.loc[result_df['return'] < -2, 'target'] = -1
result_df['target'].value_counts()

 1    629
 0    589
-1    578
Name: target, dtype: int64

In [26]:
result_df

,close,Chaikin_oscillator,CCI,williams_percentage,PSAR,MFI,Keltner_channels,TRIX,Ultimate_oscillator,ad_indicator,...,RSI,ichimoku_cloud,MACD,Stochastic_oscillator,Volume_Weighted_Average_Price,Bollinger_band,Fibonacci_retracement,ADX,return,target
timestamp,,,,,,,,,,,,,,,,,,,,,
2018-01-16 00:00:00+00:00,11282.49,-1,0,1,-1,-1,1,0,0,-1,...,0,0,-1,1,-1,0,1,-1.0,-20.786387,-1
2018-01-17 00:00:00+00:00,11162.70,1,0,0,-1,-1,1,0,0,-1,...,0,0,-1,0,-1,0,1,-1.0,-18.153803,-1
2018-01-18 00:00:00+00:00,11175.52,-1,0,0,-1,1,0,0,0,-1,...,0,0,-1,0,-1,0,1,-1.0,-18.019833,-1
2018-01-19 00:00:00+00:00,11521.76,1,0,0,-1,-1,0,0,0,1,...,1,0,-1,0,-1,0,1,-1.0,2.120720,1
2018-01-20 00:00:00+00:00,12783.94,1,0,0,-1,-1,0,0,0,1,...,1,0,-1,0,-1,0,1,-1.0,14.523726,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-31 00:00:00+00:00,28477.29,1,0,0,1,-1,0,-1,1,1,...,0,0,1,0,-1,0,1,1.0,4.408412,1
2023-04-01 00:00:00+00:00,28465.30,-1,0,0,1,-1,0,-1,1,1,...,0,0,-1,0,-1,0,1,1.0,0.385917,0
2023-04-02 00:00:00+00:00,28186.76,-1,0,0,1,-1,0,-1,1,1,...,0,0,-1,0,-1,0,1,1.0,0.532502,0


In [27]:
result_df.describe()

,close,Chaikin_oscillator,CCI,williams_percentage,PSAR,MFI,Keltner_channels,TRIX,Ultimate_oscillator,ad_indicator,...,RSI,ichimoku_cloud,MACD,Stochastic_oscillator,Volume_Weighted_Average_Price,Bollinger_band,Fibonacci_retracement,ADX,return,target
count,1796.000000,1796.000000,1796.000000,1796.000000,1796.000000,1796.000000,1796.000000,1796.000000,1796.000000,1796.000000,...,1796.000000,1796.000000,1796.000000,1796.000000,1796.000000,1796.000000,1796.000000,1796.000000,1796.000000,1796.000000
mean,20151.258641,0.031180,0.053452,-0.073497,-0.015590,-0.013363,-0.061247,0.012249,0.993318,0.218263,...,-0.036748,-0.052895,0.052339,-0.073497,-0.056793,-0.017817,0.721604,-0.013363,0.298468,0.028396
std,16453.645806,0.999792,0.648490,0.649104,1.000157,1.000189,0.535775,0.994058,0.081490,0.976162,...,0.544189,0.517103,0.998908,0.649104,0.998664,0.331646,0.692499,1.000189,6.641005,0.819522
min,3232.510000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-38.110115,-1.000000
25%,7767.707500,-1.000000,0.000000,-0.250000,-1.000000,-1.000000,0.000000,-1.000000,1.000000,-1.000000,...,0.000000,0.000000,-1.000000,-0.250000,-1.000000,0.000000,1.000000,-1.000000,-3.261746,-1.000000
50%,11269.460000,1.000000,0.000000,0.000000,-1.000000,-1.000000,0.000000,0.500000,1.000000,1.000000,...,0.000000,0.000000,1.000000,0.000000,-1.000000,0.000000,1.000000,-1.000000,0.203803,0.000000
75%,31157.840000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,3.798681,1.000000
max,67549.140000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,28.295854,1.000000


In [28]:
result_df.to_csv('indicators.csv')